In [33]:
import rasterio
import rasterio.plot
import pandas as pd
import matplotlib
import geopandas as gpd
import numpy as np
import rioxarray

# <<< FERTILISER >>>
# The dataset consists of maps of the predicted average annual application rates (2010-2015) of three different inorganic 
# chemical fertilisers – nitrogen (N), phosphorus (P) and potassium (K) – in England across a six-year period, along with 
# their respective estimates of uncertainty, at a 1 km x 1 km resolution. 
FERTILISER_K_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_k_prediction_uncertainty.tif'
FERTILISER_N_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_n_prediction_uncertainty.tif'
FERTILISER_P_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_p_prediction_uncertainty.tif'

PESTICIDE_PATH = 'Datasets\Digimap\Land-Cover-plus-Pesticides_4719434\Amidosulfuron.tif'
# dataset = rasterio.open(FERTILISER_P_PATH)
# data = dataset.read()
# np.max(data)

# Dimensions: 700000x1300000
# These datasets all only have one band
ELEVATION_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\HGHT_1km.tif'
CUMULATIVE_CATCHMENT_AREA_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\CCAR_1km.tif'
SURFACE_TYPE_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\SURF_1km.tif'
OUTFLOWING_DRAINAGE_DIRECTION_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\OUTF_1km.tif'
INFLOWING_DRAINAGE_PATTERN_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\INFL_1km.tif'

LAND_COVER_MAP_PATH = 'Datasets/Digimap/a3ff9411-3a7a-47e1-9b3e-79f21648237d/data/gb2021lcm1km_percentage_target.tif'

dataset = rioxarray.open_rasterio(ELEVATION_PATH)
dataset.name = 'data'
df = dataset.to_dataframe()

# Adding 25 to x and y coordinates
# Make this into a function for the other IHDTM datasets
df.index = df.index.set_levels(df.index.levels[1]+25, level=1)
df.index = df.index.set_levels(df.index.levels[2]+25, level=2)
df

spatial_ref  data
band y         x                          
1    1299500.0 500.0               0 -9999
               1500.0              0 -9999
               2500.0              0 -9999
               3500.0              0 -9999
               4500.0              0 -9999
...                              ...   ...
     500.0     695500.0            0 -9999
               696500.0            0 -9999
               697500.0            0 -9999
               698500.0            0 -9999
               699500.0            0 -9999

[910000 rows x 2 columns]

In [4]:
df['data'].value_counts()

0     18203710
1       123393
2        71220
3        51085
4        39528
        ...   
89        3191
93        3155
87        3143
82        3140
90        3129
Name: data, Length: 101, dtype: int64